In [1]:
import sys
import copy
import random
import os
from typing import Tuple
from collections import OrderedDict

import numpy as np
import PIL 
import matplotlib.pyplot as plt

# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
# import tensorflow as tf

import torch
from torch import nn, optim
from torch.utils import data
from torchvision import datasets, transforms

In [2]:
# sys.path.append("/home/matthias/Documents/EmbeddedAI/deep-microcompression/")
sys.path.append("../../")

from development.models.utils import convert_from_sequential_torch_to_dmc

from development import (
    Sequential,
    AvgPool2d,
    BatchNorm2d,
    Conv2d,
    Flatten,
    Linear,
    ReLU6,

    EarlyStopper,

    QuantizationGranularity,
    QuantizationScheme
)

/home/matthias/Documents/EmbeddedAI/deep-microcompression/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

input_shape = (3, 32, 32)
LUCKY_NUMBER = 25
torch.manual_seed(LUCKY_NUMBER)
torch.random.manual_seed(LUCKY_NUMBER)
torch.cuda.manual_seed(LUCKY_NUMBER)

DEVICE

'cuda'

## Getting the Datasets

In [4]:
data_transform = transforms.Compose([
    # transforms.RandomCrop((24, 24)),
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=(0.5071, 0.4867, 0.4408),
        std=(0.2675, 0.2565, 0.2761)
    )
])


cifar100_train_dataset = datasets.CIFAR100("../../../Datasets/CIFAR_100/", train=True, download=True, transform=data_transform)
cifar100_test_dataset = datasets.CIFAR100("../../../Datasets/CIFAR_100/", train=False, download=True, transform=data_transform)

cifar100_train_loader = data.DataLoader(cifar100_train_dataset, batch_size=256, shuffle=True, num_workers=os.cpu_count())
cifar100_test_loader = data.DataLoader(cifar100_test_dataset, batch_size=256, shuffle=False, num_workers=os.cpu_count())

top1_acc_fun = lambda y_pred, y_true: ((y_pred).argmax(dim=1) == y_true).sum().item()*100
top5_acc_fun = lambda y_pred, y_true: (y_pred.topk(5, dim=1).indices == y_true.unsqueeze(1)).any(dim=1).sum().item()*100

metrics = {
    "top1acc": top1_acc_fun,
    "top5acc" : top5_acc_fun
}

In [10]:
pretraind_vqq_13_torch = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_vgg13_bn", pretrained=True)
vgg_13_torch_model = pretraind_vqq_13_torch.features + nn.Sequential(nn.Flatten()) + pretraind_vqq_13_torch.classifier
vgg_13_dmc_model = convert_from_sequential_torch_to_dmc(vgg_13_torch_model).to(DEVICE)


Using cache found in /home/matthias/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


## Original Model

In [12]:
vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics=metrics, device=DEVICE)

100%|██████████| 40/40 [00:08<00:00,  4.71it/s]


{'top1acc': 74.63, 'top5acc': 91.09}

In [7]:
# vgg_13_dmc_model = vgg_13_dmc_model.fuse()

# evaluate_result = vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics=metrics, device=DEVICE)
# original_size = vgg_13_dmc_model.get_size_in_bytes()
# print(f"The original model accuracy is top1acc = {evaluate_result["top1acc"]:.2f}%  top5acc = {evaluate_result["top5acc"]:.2f}% with size {original_size} bytes.")



In [8]:
# fused_vgg_13_dmc_model = vgg_13_dmc_model.fuse().to(DEVICE)

# evaluate_result = fused_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics=metrics, device=DEVICE)
# original_size = fused_vgg_13_dmc_model.get_size_in_bytes()
# print(f"The fused model accuracy is top1acc = {evaluate_result["top1acc"]:.2f}%  top5acc = {evaluate_result["top5acc"]:.2f}% with size {original_size} bytes.")


In [ ]:
# fused_vgg_13_dmc_model = vgg_13_dmc_model.fuse().to(DEVICE)

# evaluate_result = fused_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics=metrics, device=DEVICE)
# original_size = fused_vgg_13_dmc_model.get_size_in_bytes()
# print(f"The original model accuracy is {evaluate_result["acc"]*100:.2f}% with size {original_size} bytes.")

# # fused_vgg_13_dmc_model.convert_to_c(var_name="vgg_13_dmc_model", src_dir="./Arduino Nano 33 BLE/src/", include_dir="./Arduino Nano 33 BLE/include/")
# fused_vgg_13_dmc_model.convert_to_c(input_shape=input_shape, var_name="vgg_13_dmc_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/")

# print(fused_vgg_13_dmc_model.test(device=DEVICE, var_name="vgg_13_dmc_model", src_dir="./HP HP Pavilion Laptop 15-cs3xxx/src/", include_dir="./HP HP Pavilion Laptop 15-cs3xxx/include/"))
# # print(fused_vgg_13_dmc_model(vgg_13_dmc_model.test_input[0].unsqueeze(dim=0).detach().clone()))
# evaluate_result["acc"], original_size

In [ ]:
early_stopper = EarlyStopper(
    metric_name="validation_loss",
    min_valid_diff=1e-4,
    mode="min",
    patience=2,
    restore_best_state_dict=True,
)

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.SGD(vgg_13_dmc_model.parameters(), lr=1.e-3)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=2)

sensitivity = vgg_13_dmc_model.get_layers_prune_channel_sensity_(
    input_shape=input_shape, data_loader=cifar100_test_loader, metrics=metrics, device=DEVICE,
    train=False,
    train_dataloader=cifar100_train_loader, epochs=2, criterion_fun=criterion_fun, optimizer_fun=optimizion_fun,
    lr_scheduler=lr_scheduler, 
)

 88%|████████▊ | 35/40 [00:06<00:00,  5.59it/s]

In [ ]:
plt.figure(figsize=(10, 6))
for layer, values in sensitivity['acc'].items():
    print(len(values), values)
    sparsity, acc = zip(*values)
    plt.plot(sparsity, acc, marker='.', label=layer+f"_{str(len(values))}")

plt.xlabel("Sparsity Level")
plt.ylabel("Accuracy (%)")
plt.title("Lenet-5 model Sparsity Sensitivity per Layer")
plt.legend()
plt.grid(False)

list(plt.gca().spines.values())[0].set_visible(False)
list(plt.gca().spines.values())[1].set_visible(False)
list(plt.gca().spines.values())[3].set_visible(False)

plt.savefig("Lenet-5 model Sparsity Sensitivity per Layer", dpi=300, bbox_inches="tight")
plt.show()



## Testing Compression on VGG13

## Pruning

In [ ]:
for i in range(0, 11, 1):
    sp = i/10
    compression_config = {
        "prune_channel" :{
            "sparsity" : sp,
            "metric" : "l2"
        },
    }
    compressed_vgg_13_dmc_model = vgg_13_dmc_model.init_compress(compression_config, input_shape=input_shape)

    evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
    size = compressed_vgg_13_dmc_model.get_size_in_bytes()
    print(f"Before training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")


  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:40<00:00,  1.01s/it]


Before training, sparsity = 0.0 acc1 = 74.63% acc5 = 91.09% size =  100.0000


100%|██████████| 40/40 [00:43<00:00,  1.08s/it]


Before training, sparsity = 0.1 acc1 = 44.80% acc5 = 65.40% size =   81.2115


...

Before training, sparsity = 0.8 acc1 = 1.00% acc5 = 5.00% size =    4.1533


100%|██████████| 40/40 [00:35<00:00,  1.13it/s]


Before training, sparsity = 0.9 acc1 = 1.00% acc5 = 5.00% size =    1.0858


100%|██████████| 40/40 [00:36<00:00,  1.11it/s]


Before training, sparsity = 1.0 acc1 = 1.00% acc5 = 5.00% size =    0.0032


### Dynamic Quantization Per Tensor

In [ ]:
for b in [8, 4, 2]:
    compression_config = {
        "quantize" : {
            "scheme" : QuantizationScheme.DYNAMIC,
            "granularity": QuantizationGranularity.PER_TENSOR,
            "bitwidth" : b
        }
    }
    compressed_vgg_13_dmc_model = vgg_13_dmc_model.init_compress(compression_config, input_shape=input_shape)

    evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
    size = compressed_vgg_13_dmc_model.get_size_in_bytes()
    print(f"Before training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")


  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:44<00:00,  1.11s/it]


Before training, sparsity = 1.0 acc1 = 74.43% acc5 = 90.94% size =   25.0307


100%|██████████| 40/40 [00:44<00:00,  1.11s/it]


Before training, sparsity = 1.0 acc1 = 45.30% acc5 = 67.25% size =   12.5358


100%|██████████| 40/40 [00:41<00:00,  1.04s/it]


Before training, sparsity = 1.0 acc1 = 1.00% acc5 = 5.00% size =    6.2883


### Dynamic Quantization Per Channel

In [ ]:
for b in [8, 4, 2]:
    compression_config = {
        "quantize" : {
            "scheme" : QuantizationScheme.DYNAMIC,
            "granularity": QuantizationGranularity.PER_CHANNEL,
            "bitwidth" : b
        }
    }
    compressed_vgg_13_dmc_model = vgg_13_dmc_model.init_compress(compression_config, input_shape=input_shape)

    evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
    size = compressed_vgg_13_dmc_model.get_size_in_bytes()
    print(f"Before training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")


100%|██████████| 40/40 [00:38<00:00,  1.05it/s]


Before training, sparsity = 1.0 acc1 = 74.66% acc5 = 91.06% size =   25.0713


100%|██████████| 40/40 [00:37<00:00,  1.07it/s]


Before training, sparsity = 1.0 acc1 = 72.38% acc5 = 90.03% size =   12.5764


100%|██████████| 40/40 [00:37<00:00,  1.06it/s]


Before training, sparsity = 1.0 acc1 = 1.00% acc5 = 5.09% size =    6.3290


### Static Quantization Per Tensor

In [ ]:
for b in [8, 4, 2]:
    compression_config = {
        "quantize" : {
            "scheme" : QuantizationScheme.STATIC,
            "granularity": QuantizationGranularity.PER_TENSOR,
            "bitwidth" : b
        }
    }
    compressed_vgg_13_dmc_model = vgg_13_dmc_model.init_compress(compression_config, input_shape=input_shape, calibration_data=next(iter(cifar100_test_loader))[0])

    evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
    size = compressed_vgg_13_dmc_model.get_size_in_bytes()
    print(f"Before training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")


100%|██████████| 40/40 [01:02<00:00,  1.57s/it]


Before training, sparsity = 1.0 acc1 = 74.28% acc5 = 90.90% size =   25.0309


100%|██████████| 40/40 [00:59<00:00,  1.50s/it]


Before training, sparsity = 1.0 acc1 = 27.19% acc5 = 44.92% size =   12.5360


100%|██████████| 40/40 [00:57<00:00,  1.43s/it]


Before training, sparsity = 1.0 acc1 = 1.00% acc5 = 5.00% size =    6.2885


### Static Quantization Per Channel

In [ ]:
for b in [8, 4, 2]:
    compression_config = {
        "quantize" : {
            "scheme" : QuantizationScheme.STATIC,
            "granularity": QuantizationGranularity.PER_CHANNEL,
            "bitwidth" : b
        }
    }
    compressed_vgg_13_dmc_model = vgg_13_dmc_model.init_compress(compression_config, input_shape=input_shape,calibration_data=next(iter(cifar100_test_loader))[0])

    evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
    size = compressed_vgg_13_dmc_model.get_size_in_bytes()
    print(f"Before training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")


100%|██████████| 40/40 [00:54<00:00,  1.37s/it]


Before training, sparsity = 1.0 acc1 = 74.64% acc5 = 91.12% size =   25.0715


100%|██████████| 40/40 [00:53<00:00,  1.35s/it]


Before training, sparsity = 1.0 acc1 = 59.75% acc5 = 78.69% size =   12.5766


100%|██████████| 40/40 [00:52<00:00,  1.32s/it]


Before training, sparsity = 1.0 acc1 = 1.09% acc5 = 5.13% size =    6.3292


## Training with Compression

### Pruning

In [ ]:
sp = 1/10
compression_config = {
    "prune_channel" :{
        "sparsity" : sp,
        "metric" : "l2"
    },
}
compressed_vgg_13_dmc_model = vgg_13_dmc_model.init_compress(compression_config, input_shape=input_shape)

evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
size = compressed_vgg_13_dmc_model.get_size_in_bytes()
print(f"Before training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.SGD(compressed_vgg_13_dmc_model.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=3)

early_stopper = EarlyStopper(
    metric_name="validation_loss",
    min_valid_diff=1e-5,
    mode="min",
    patience=7,
    restore_best_state_dict=True,
)

compressed_vgg_13_dmc_model.fit(
    cifar100_train_loader, 
    52000,
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=cifar100_test_loader,
    metrics=metrics,
    verbose=True, 
    callbacks=[early_stopper]
)

evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
print(f"After training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")


  0%|          | 0/40 [00:00<?, ?it/s]

100%|██████████| 40/40 [00:41<00:00,  1.04s/it]


Before training, sparsity = 0.1 acc1 = 44.80% acc5 = 65.40% size =   81.2115


  0%|          | 1/52000 [10:30<9108:12:31, 630.58s/it]

epoch    0 | train loss 0.0004 | validation loss 0.0052 | train top1acc 97.4200 | validation top1acc 70.8700 | train top5acc 99.6200 | validation top5acc 89.0100


...

epoch    6 | train loss 0.0000 | validation loss 0.0053 | train top1acc 99.9780 | validation top1acc 71.7700 | train top5acc 100.0000 | validation top5acc 89.5900


  0%|          | 7/52000 [1:22:17<10186:52:03, 705.34s/it]


epoch    7 | train loss 0.0000 | validation loss 0.0053 | train top1acc 99.9780 | validation top1acc 71.6800 | train top5acc 100.0000 | validation top5acc 89.6000
Stopping Training of Sequential with at 0 epoch with best validation_loss = 0.005220910534262657


100%|██████████| 40/40 [00:38<00:00,  1.03it/s]

After training, sparsity = 0.1 acc1 = 71.68% acc5 = 89.60% size =   81.2115


In [ ]:
sp = 3/10
compression_config = {
    "prune_channel" :{
        "sparsity" : sp,
        "metric" : "l2"
    },
}
compressed_vgg_13_dmc_model = vgg_13_dmc_model.init_compress(compression_config, input_shape=input_shape)

evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
size = compressed_vgg_13_dmc_model.get_size_in_bytes()
print(f"Before training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.SGD(compressed_vgg_13_dmc_model.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=3)

early_stopper = EarlyStopper(
    metric_name="validation_loss",
    min_valid_diff=1e-5,
    mode="min",
    patience=7,
    restore_best_state_dict=True,
)

compressed_vgg_13_dmc_model.fit(
    cifar100_train_loader, 
    50,
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=cifar100_test_loader,
    metrics=metrics,
    verbose=True, 
    callbacks=[early_stopper]
)

evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
print(f"After training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")


100%|██████████| 40/40 [00:41<00:00,  1.03s/it]


Before training, sparsity = 0.3 acc1 = 4.07% acc5 = 12.53% size =   49.3318


  2%|▏         | 1/50 [10:40<8:43:01, 640.43s/it]

epoch    0 | train loss 0.0070 | validation loss 0.0066 | train top1acc 60.8940 | validation top1acc 61.2000 | train top5acc 79.6760 | validation top5acc 82.2400


  4%|▍         | 2/50 [21:01<8:23:25, 629.28s/it]

...

epoch    8 | train loss 0.0001 | validation loss 0.0063 | train top1acc 99.8060 | validation top1acc 68.4300 | train top5acc 99.9900 | validation top5acc 87.6900


 18%|█▊        | 9/50 [1:41:41<7:43:15, 677.93s/it]


epoch    9 | train loss 0.0001 | validation loss 0.0063 | train top1acc 99.8240 | validation top1acc 68.4700 | train top5acc 99.9900 | validation top5acc 87.6700
Stopping Training of Sequential with at 2 epoch with best validation_loss = 0.005783408880233765


100%|██████████| 40/40 [00:39<00:00,  1.01it/s]

After training, sparsity = 0.3 acc1 = 68.47% acc5 = 87.67% size =   49.3318


In [ ]:
b = 4
compression_config = {
"quantize" : {
    "scheme" : QuantizationScheme.DYNAMIC,
    "granularity": QuantizationGranularity.PER_TENSOR,
    "bitwidth" : b
}
}
compressed_vgg_13_dmc_model = vgg_13_dmc_model.init_compress(compression_config, input_shape=input_shape)

evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
size = compressed_vgg_13_dmc_model.get_size_in_bytes()
print(f"Before training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")

criterion_fun = nn.CrossEntropyLoss()
optimizion_fun = optim.SGD(compressed_vgg_13_dmc_model.parameters(), lr=1e-3, momentum=0.9, weight_decay=5e-4)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizion_fun, mode="min", patience=3)

early_stopper = EarlyStopper(
    metric_name="validation_loss",
    min_valid_diff=1e-5,
    mode="min",
    patience=7,
    restore_best_state_dict=True,
)

compressed_vgg_13_dmc_model.fit(
    cifar100_train_loader, 
    50,
    criterion_fun, optimizion_fun, lr_scheduler,
    validation_dataloader=cifar100_test_loader,
    metrics=metrics,
    verbose=True, 
    callbacks=[early_stopper]
)

evaluate_result = compressed_vgg_13_dmc_model.evaluate(cifar100_test_loader, metrics, device=DEVICE)
print(f"After training, sparsity = {sp} acc1 = {evaluate_result["top1acc"]:.2f}% acc5 = {evaluate_result["top5acc"]:.2f}% size = {size/original_size*100:9.4f}")


100%|██████████| 40/40 [00:37<00:00,  1.08it/s]


Before training, sparsity = 0.3 acc1 = 45.30% acc5 = 67.25% size =   12.5358


  2%|▏         | 1/50 [09:16<7:34:26, 556.46s/it]

epoch    0 | train loss 0.0005 | validation loss 0.0056 | train top1acc 97.1360 | validation top1acc 71.0400 | train top5acc 99.5240 | validation top5acc 89.3000


  4%|▍         | 2/50 [18:38<7:27:46, 559.72s/it]

...

epoch    6 | train loss 0.0000 | validation loss 0.0056 | train top1acc 99.9700 | validation top1acc 72.4500 | train top5acc 100.0000 | validation top5acc 90.1900


 14%|█▍        | 7/50 [1:13:47<7:33:20, 632.57s/it]


epoch    7 | train loss 0.0000 | validation loss 0.0057 | train top1acc 99.9760 | validation top1acc 72.2900 | train top5acc 100.0000 | validation top5acc 89.9000
Stopping Training of Sequential with at 0 epoch with best validation_loss = 0.00564540596678853


100%|██████████| 40/40 [00:35<00:00,  1.14it/s]


After training, sparsity = 0.3 acc1 = 72.29% acc5 = 89.90% size =   12.5358
